In [1]:
import yaml
import os
import pathlib
import json

In [2]:
refdict = yaml.load(open('BH76_ref_energies.yaml','r'), Loader=yaml.Loader)
kcal = 627.509
all_mols = {}

In [3]:
def process_files(input_dir):
    mad_energies = {}
    # Iterate through the directory and its subdirectories
    for root, dirs, files in os.walk(input_dir):
        for dir in dirs:
            mol_name = dir
            all_mols[mol_name] = True

        for file in files:
            if file == "struc.calc_info.json": 
                
                input_file_path = os.path.join(root, file)
                # pbe and pbescf differ in path (geometry folder..) need to adjust this
                mol_name = input_file_path.split('/')[5]

                f = open(input_file_path)
                data = json.load(f)
                #print(mol_name, data['return_energy'])
                mad_energies[mol_name] = data['return_energy']
    
    return mad_energies

In [4]:
path = os.path.abspath(os.getcwd())
input_directory = path+'/pbe'  # Change this to your desired input directory
mad_pbehf = process_files(input_directory)

In [6]:
# just replace 5 by 6 in def above, ugly... but allows to run current version in one go
def process_files(input_dir):
    mad_energies = {}
    # Iterate through the directory and its subdirectories
    for root, dirs, files in os.walk(input_dir):
        for dir in dirs:
            mol_name = dir
            all_mols[mol_name] = True

        for file in files:
            if file == "struc.calc_info.json": 
                
                input_file_path = os.path.join(root, file)
                # pbe and pbescf differ in path (geometry folder..) need to adjust this
                mol_name = input_file_path.split('/')[6]

                f = open(input_file_path)
                data = json.load(f)
                #print(mol_name, data['return_energy'])
                mad_energies[mol_name] = data['return_energy']
    
    return mad_energies

In [7]:
mad_pbescf = process_files(path+'/pbe_scf/geometries')

In [8]:
print(len(mad_pbehf.keys()))
print(len(mad_pbescf.keys()))

86
87


In [9]:
print('MISSING: ')
cmdict = yaml.load(open('BH76_chg_2s.yaml','r'), Loader=yaml.Loader)
for i in all_mols.keys():
    if i in mad_pbescf.keys():
        pass
    else:
        #print(i, "\t", cmdict[i])
        print(i)

MISSING: 
.ipynb_checkpoints


In [10]:
#print(refdict[1])

In [11]:
# only for debug use
h_pbe = -0.4999410227691473
h_hf  = -0.4999535118271135

In [12]:
# iterate over Refdict, stoich keys lookup in madness energies dict, calc bh according to stoich, scale to kcal
#
# !! code does not support restart from h single electron wf, pbe@hf for h does not work
# here, TEMPORARILY, use hf/pbe energy instead
pbe_bh76_mad = {}
mad_pbehf['h'] = h_hf
for bh in refdict.keys():
    e_bh = 0   
    for system in refdict[bh]['Stoich'].keys():
        stoich_fac = refdict[bh]['Stoich'][system]
        e_bh += stoich_fac * mad_pbehf[system]
        #print(e_bh)

    pbe_bh76_mad[bh] = e_bh * kcal
    #print(bh, pbe_bh76_mad[bh])

In [13]:
pbescf_bh76_mad = {}
for bh in refdict.keys():
    e_bh = 0   
    for system in refdict[bh]['Stoich'].keys():
        stoich_fac = refdict[bh]['Stoich'][system]
        e_bh += stoich_fac * mad_pbescf[system]
        #print(e_bh)

    pbescf_bh76_mad[bh] = e_bh * kcal
    #print(bh, pbe_bh76_mad[bh])

In [14]:
print(37, pbe_bh76_mad[37], 48.10 - pbe_bh76_mad[37])

37 47.62230232638348 0.47769767361651816


In [15]:
ref_bh, mad_pbehf_bh,mad_pbescf_bh = [], [], []
for i in refdict.keys():
    #print(i,'\t', "%.2f" % pbe_bh76_mad[i], '\t\t', refdict[i]['Ref'])
    mad_pbehf_bh.append(pbe_bh76_mad[i])
    mad_pbescf_bh.append(pbescf_bh76_mad[i])
    ref_bh.append(refdict[i]['Ref'])

In [16]:
error_bh = []
error_bhscf = []

for i in pbe_bh76_mad.keys():
    error_bh.append(pbe_bh76_mad[i] - refdict[i]['Ref'])
    error_bhscf.append(pbescf_bh76_mad[i] - refdict[i]['Ref'])
    #print(i, pbe_bh76_mad[i] - refdict[i]['Ref'])
#print(error)

In [17]:
from scipy import stats
print(stats.median_abs_deviation(error_bh))
print(stats.median_abs_deviation(error_bhscf))

2.5227178776254644
2.6936794610096655


In [18]:
from statistics import mean
print(mean(error_bh))
print(mean(error_bhscf))

-0.7150351323570919
-8.707904125274554


In [19]:
#for i in error_bhscf:
#    print(i)

In [43]:
import pandas as pd

In [44]:
reactions = open('reactions.txt', 'r')
rct_string = reactions.readlines()

In [45]:
a = []
for i in rct_string:
    a.append(i.replace("\n", ""))
for i in range(len(a)):
    if i % 2 == 0:
        a[i] = "\ce{"+a[i]+"}"

In [46]:
#mad_pbehf_bh
#mad_pbescf_bh
table1 = pd.DataFrame(list(zip(a, ref_bh, error_bhscf, error_bh)), columns = ['Reaction', 'Exact', 'SCF', 'HF-DFT'], index=refdict.keys())
table1['SCF'] = table1['SCF'].apply(lambda x: round(x, 2))
table1['HF-DFT'] = table1['HF-DFT'].apply(lambda x: round(x, 2))


In [47]:
print(table1)

                                            Reaction  Exact    SCF  HF-DFT
1                         \ce{H + N 2 O -> N 2 + OH}   17.7  -7.61    4.21
2                                          1 reverse   82.6 -29.76   -9.98
3                              \ce{H + HF -> HF + H}   42.1 -15.06  -11.26
4                                          3 reverse   42.1 -15.06  -11.26
5                            \ce{H + HCl -> HCl + H}   17.8  -8.18   -3.73
..                                               ...    ...    ...     ...
72                                        71 reverse   17.8  -7.48   -1.54
73                   \ce{NH 2 + CH 4 -> NH 3 + CH 3}   13.9  -9.24   -3.99
74                                        73 reverse   16.9  -8.89   -2.83
75  \ce{s-trans cis-C 5 H 8 -> s-trans cis-C 5 H 8 }   39.7  -8.49  -10.19
76                                        75 reverse   39.7  -8.49  -10.19

[76 rows x 4 columns]


In [48]:
print(table1.to_latex(float_format="%.2f"))

\begin{tabular}{llrrr}
\toprule
 & Reaction & Exact & SCF & HF-DFT \\
\midrule
1 & \ce{H + N 2 O -> N 2 + OH} & 17.70 & -7.61 & 4.21 \\
2 & 1 reverse & 82.60 & -29.76 & -9.98 \\
3 & \ce{H + HF -> HF + H} & 42.10 & -15.06 & -11.26 \\
4 & 3 reverse & 42.10 & -15.06 & -11.26 \\
5 & \ce{H + HCl -> HCl + H} & 17.80 & -8.18 & -3.73 \\
6 & 5 reverse & 17.80 & -8.18 & -3.73 \\
7 & \ce{CH 3 F + H -> CH 3 + HF} & 30.50 & -12.01 & -5.93 \\
8 & 7 reverse & 56.90 & -15.75 & -6.61 \\
9 & \ce{F 2 + H -> F + HF} & 1.50 & -10.73 & 24.85 \\
10 & 9 reverse & 104.80 & -24.98 & 14.88 \\
11 & \ce{CH 3 + ClF -> CH 3 F + Cl} & 7.10 & -12.41 & 6.62 \\
12 & 11 reverse & 59.80 & -18.36 & 2.20 \\
13 & \ce{CH 3 F + F - -> FCH 3 + F -} & -0.60 & -4.66 & 3.82 \\
14 & 13 reverse & -0.60 & -4.66 & 3.82 \\
15 & \ce{CH 3 F . . . F - -> FCH 3 . . . F -} & 13.40 & -5.87 & 1.29 \\
16 & 15 reverse & 13.40 & -5.87 & 1.29 \\
17 & \ce{CH 3 Cl + Cl - -> ClCH 3 + Cl -} & 2.50 & -5.51 & 3.11 \\
18 & 17 reverse & 2.50 & -5.51 & 3.

In [63]:
scf_ref = open('lcao_bh_scf_pbe.txt', 'r')
scfref_string = scf_ref.readlines()
lcao_scf = []
for i in scfref_string:
    lcao_scf.append(float(i.replace("\n","")))

In [71]:
hfdft_ref = open('lcao_bh_hfdft_pbe.txt', 'r')
hfdftref_string = hfdft_ref.readlines()
lcao_hfdft = []
for i in hfdftref_string:
    lcao_hfdft.append(float(i.replace("\n","")))

In [87]:
scf_diff = []
for i,j in zip(lcao_scf,error_bhscf):
    scf_diff.append(round((i-j),2))
print(min(scf_diff), max(scf_diff))

-1.44 0.34


In [88]:
hfdft_diff = []
for i,j in zip(lcao_hfdft,error_bh):
    hfdft_diff.append(round((i-j),2))
print(min(hfdft_diff), max(hfdft_diff))

-1.0 0.22


In [73]:
error_diff = pd.DataFrame(list(zip(a, scf_diff, hfdft_diff)), columns = ['Reaction', 'SCF', 'HF-DFT'], index=refdict.keys())

In [74]:
print(error_diff)

                                            Reaction   SCF  HF-DFT
1                         \ce{H + N 2 O -> N 2 + OH} -0.14   -0.32
2                                          1 reverse -0.03    0.15
3                              \ce{H + HF -> HF + H} -0.09   -0.42
4                                          3 reverse -0.09   -0.42
5                            \ce{H + HCl -> HCl + H}  0.34    0.11
..                                               ...   ...     ...
72                                        71 reverse -0.14   -0.10
73                   \ce{NH 2 + CH 4 -> NH 3 + CH 3} -0.13   -0.08
74                                        73 reverse -0.18   -0.10
75  \ce{s-trans cis-C 5 H 8 -> s-trans cis-C 5 H 8 } -0.04   -0.09
76                                        75 reverse -0.04   -0.09

[76 rows x 3 columns]


In [77]:
print(error_diff.to_latex(float_format="%.2f"))

\begin{tabular}{llrr}
\toprule
 & Reaction & SCF & HF-DFT \\
\midrule
1 & \ce{H + N 2 O -> N 2 + OH} & -0.14 & -0.32 \\
2 & 1 reverse & -0.03 & 0.15 \\
3 & \ce{H + HF -> HF + H} & -0.09 & -0.42 \\
4 & 3 reverse & -0.09 & -0.42 \\
5 & \ce{H + HCl -> HCl + H} & 0.34 & 0.11 \\
6 & 5 reverse & 0.34 & 0.11 \\
7 & \ce{CH 3 F + H -> CH 3 + HF} & -0.22 & -0.49 \\
8 & 7 reverse & -0.14 & -0.11 \\
9 & \ce{F 2 + H -> F + HF} & -0.27 & -0.75 \\
10 & 9 reverse & -0.12 & -0.40 \\
11 & \ce{CH 3 + ClF -> CH 3 F + Cl} & -0.62 & -0.57 \\
12 & 11 reverse & 0.33 & 0.22 \\
13 & \ce{CH 3 F + F - -> FCH 3 + F -} & -0.66 & -0.17 \\
14 & 13 reverse & -0.66 & -0.17 \\
15 & \ce{CH 3 F . . . F - -> FCH 3 . . . F -} & -0.13 & -0.07 \\
16 & 15 reverse & -0.13 & -0.07 \\
17 & \ce{CH 3 Cl + Cl - -> ClCH 3 + Cl -} & -0.41 & -0.41 \\
18 & 17 reverse & -0.41 & -0.41 \\
19 & \ce{CH 3 Cl . . . Cl - -> ClCH 3 . . . Cl -} & -0.31 & -0.29 \\
20 & 19 reverse & -0.31 & -0.29 \\
21 & \ce{CH 3 Cl + F - -> CH 3 F + Cl -} & -0.92 

In [83]:
scf_winner = []
lcao_counter = 0
mra_counter = 0
for i,j in zip(lcao_scf,error_bhscf):
    if abs(i) < abs(j):
        scf_winner.append('LCAO')
        lcao_counter +=1
    elif abs(i) > abs(j):
        scf_winner.append('MRA')
        mra_counter +=1
    else:
        scf_winner.append('draw')

In [84]:
print(scf_winner)
print(lcao_counter, mra_counter)

['MRA', 'MRA', 'MRA', 'MRA', 'LCAO', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'LCAO', 'MRA', 'LCAO', 'MRA', 'MRA', 'LCAO', 'LCAO', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA']
14 62


In [85]:
hf_dft_winner = []
lcao_counter = 0
mra_counter = 0
for i,j in zip(lcao_hfdft,error_bh):
    if abs(i) < abs(j):
        hf_dft_winner.append('LCAO')
        lcao_counter +=1
    elif abs(i) > abs(j):
        hf_dft_winner.append('MRA')
        mra_counter +=1
    else:
        hf_dft_winner.append('draw')

In [86]:
print(hf_dft_winner)
print(lcao_counter, mra_counter)

['LCAO', 'LCAO', 'MRA', 'MRA', 'LCAO', 'LCAO', 'MRA', 'MRA', 'LCAO', 'LCAO', 'LCAO', 'MRA', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'MRA', 'MRA', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'LCAO', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'LCAO', 'LCAO', 'MRA', 'LCAO', 'MRA', 'LCAO', 'LCAO', 'LCAO', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'LCAO', 'MRA', 'LCAO', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA', 'MRA']
40 36


In [100]:
table2 = pd.DataFrame(list(zip(a, scf_diff, hfdft_diff, scf_winner, hf_dft_winner)), columns = ['Reaction', 'SCF', 'HF-DFT', 'SCF winner', 'HF-DFT winner' ], index=refdict.keys())
table2['SCF'] = table2['SCF'].apply(lambda x: round(x, 2))
table2['HF-DFT'] = table2['HF-DFT'].apply(lambda x: round(x, 2))

In [101]:
print(table2)

                                            Reaction   SCF  HF-DFT SCF winner  \
1                         \ce{H + N 2 O -> N 2 + OH} -0.14   -0.32        MRA   
2                                          1 reverse -0.03    0.15        MRA   
3                              \ce{H + HF -> HF + H} -0.09   -0.42        MRA   
4                                          3 reverse -0.09   -0.42        MRA   
5                            \ce{H + HCl -> HCl + H}  0.34    0.11       LCAO   
..                                               ...   ...     ...        ...   
72                                        71 reverse -0.14   -0.10        MRA   
73                   \ce{NH 2 + CH 4 -> NH 3 + CH 3} -0.13   -0.08        MRA   
74                                        73 reverse -0.18   -0.10        MRA   
75  \ce{s-trans cis-C 5 H 8 -> s-trans cis-C 5 H 8 } -0.04   -0.09        MRA   
76                                        75 reverse -0.04   -0.09        MRA   

   HF-DFT winner  
1       

In [102]:
print(table2.to_latex(float_format="%.2f"))

\begin{tabular}{llrrll}
\toprule
 & Reaction & SCF & HF-DFT & SCF winner & HF-DFT winner \\
\midrule
1 & \ce{H + N 2 O -> N 2 + OH} & -0.14 & -0.32 & MRA & LCAO \\
2 & 1 reverse & -0.03 & 0.15 & MRA & LCAO \\
3 & \ce{H + HF -> HF + H} & -0.09 & -0.42 & MRA & MRA \\
4 & 3 reverse & -0.09 & -0.42 & MRA & MRA \\
5 & \ce{H + HCl -> HCl + H} & 0.34 & 0.11 & LCAO & LCAO \\
6 & 5 reverse & 0.34 & 0.11 & LCAO & LCAO \\
7 & \ce{CH 3 F + H -> CH 3 + HF} & -0.22 & -0.49 & MRA & MRA \\
8 & 7 reverse & -0.14 & -0.11 & MRA & MRA \\
9 & \ce{F 2 + H -> F + HF} & -0.27 & -0.75 & MRA & LCAO \\
10 & 9 reverse & -0.12 & -0.40 & MRA & LCAO \\
11 & \ce{CH 3 + ClF -> CH 3 F + Cl} & -0.62 & -0.57 & MRA & LCAO \\
12 & 11 reverse & 0.33 & 0.22 & LCAO & MRA \\
13 & \ce{CH 3 F + F - -> FCH 3 + F -} & -0.66 & -0.17 & MRA & LCAO \\
14 & 13 reverse & -0.66 & -0.17 & MRA & LCAO \\
15 & \ce{CH 3 F . . . F - -> FCH 3 . . . F -} & -0.13 & -0.07 & MRA & LCAO \\
16 & 15 reverse & -0.13 & -0.07 & MRA & LCAO \\
17 & \ce{CH 